In [1]:
using CSV, DataFrames, Pkg, CategoricalArrays

In [2]:
X_train_default = CSV.read("../Data/X_train_default.csv", DataFrame)
X_test_default = CSV.read("../Data/X_test_default.csv", DataFrame)
y_train_default = CSV.read("../Data/y_train_default.csv", DataFrame)
y_test_default = CSV.read("../Data/y_test_default.csv", DataFrame)
y_train_default = y_train_default[!,"TARGET"]
y_test_default = y_test_default[!,"TARGET"]
X_train__default = select(X_train_default, Not(["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]))
X_test__default = select(X_test_default, Not(["EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3"]))

Row,Column1,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,IS_EXT_SOURCE_1,IS_EXT_SOURCE_2,IS_EXT_SOURCE_3,YEAR_OF_BIRTH,YEAR_OF_EMPLOYMENT,YEAR_OF_REGISTRATION,YEAR_OF_ID_PUBLISH,YEAR_OF_PHONE_CHANGE,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,⋯
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Bool,Bool,Bool,Int64,Int64,Int64,Int64,Int64,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,Bool,⋯
1,179982,308574,0,270000.0,450000.0,24543.0,450000.0,0.003541,4.0,1,1,1,1,1,0,2.0,1,1,15,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,false,false,false,1992,2019,2016,2019,2016,true,false,false,false,true,false,true,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,false,true,false,false,true,false,false,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,⋯
2,237114,374640,0,54000.0,225000.0,11074.5,225000.0,0.020246,-1.0,1,1,1,1,0,0,2.0,3,3,8,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,false,false,false,1993,2015,2018,2014,2016,true,false,false,true,false,false,true,false,false,false,false,false,false,true,false,false,false,false,false,false,true,false,true,false,false,false,false,true,false,false,false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,false,false,⋯
3,245540,384153,0,135000.0,677664.0,43308.0,585000.0,0.003069,-1.0,1,1,0,1,0,0,2.0,3,3,10,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,false,false,false,1969,2015,2004,2019,2017,false,true,false,true,false,false,true,fa

### OCT_regression

In [3]:
grid_oct_r = IAI.GridSearch(
    IAI.OptimalTreeClassifier(
        random_seed=1,
        minbucket=10,
        refit_learner=IAI.GridSearch(
            IAI.OptimalFeatureSelectionClassifier(),
            sparsity=1:3,
        ),
    ),
    max_depth=1:4,
)

GridSearch - Unfitted OptimalTreeClassifier:
  minbucket:     10
  refit_learner: GridSearch - Unfitted OptimalFeatureSelectionClassifier

GridSearch Params:
  (sparsity=1,)
  (sparsity=2,)
  (sparsity=3,)
  random_seed:   1

GridSearch Params:
  (max_depth=1,)
  (max_depth=2,)
  (max_depth=3,)
  (max_depth=4,)

In [ ]:
IAI.fit!(grid_oct_r, X_train_default, y_train_default)

[ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
┌ Warning: Interpretable AI license expires soon: 2023-12-31T00:00:00. If you need to renew, please send us the following machine ID:
└ 6188fbeb8fca0fc6575c4f1bde01600265a480b47224e2af355dc5fa83100b9d


In [ ]:
IAI.score(grid_oct_r,X_test_default, y_test_default,criterion=:auc)

### OCT

In [ ]:
grid_oct = IAI.GridSearch(
    IAI.OptimalTreeClassifier(
        random_seed=1),
    max_depth=1:7,
    minbucket=10
)

In [ ]:
IAI.fit!(grid_oct, X_train_default, y_train_default)

In [ ]:
IAI.score(grid_oct,X_test_default, y_test_default,criterion=:auc)

### ORT

In [ ]:
grid_ort = IAI.GridSearch(
    IAI.OptimalTreeRegressor(
        random_seed=123,
    ),
    max_depth=1:5,
)

In [ ]:
IAI.fit!(grid_ort, X_train_default, y_train_default)

In [ ]:
IAI.score(grid_ort,X_test_default, y_test_default,criterion=:mse)

### ORT_RL

In [ ]:
grid_ort_rl = IAI.GridSearch(
    IAI.OptimalTreeRegressor(
        random_seed=123,
        max_depth=4,
        regression_features=All(),
    ),
    regression_lambda=[0.005, 0.01, 0.05],
)

In [ ]:
IAI.fit!(grid_ort_rl, X_train_default, y_train_default)

In [ ]:
IAI.score(grid_ort,X_test_default, y_test_default,criterion=:mse)

### Random Forest

In [ ]:
rf = IAI.RandomForestClassifier(random_seed=1, num_trees = 50)
grid_rf = IAI.GridSearch(rf, max_depth=1:4)

In [ ]:
IAI.fit!(grid_rf, X_train_default, y_train_default)

In [ ]:
IAI.score(grid_rf,X_test_default, y_test_default,criterion=:auc)

### XGBoost

In [ ]:
xgb = IAI.XGBoostClassifier(random_seed=1, num_estimators=50)
grid_xgb = IAI.GridSearch(xgb, max_depth=1:4)

In [ ]:
IAI.fit!(grid_xgb, X_train_default, y_train_default)

In [ ]:
IAI.score(grid_xgb,X_test_default, y_test_default,criterion=:auc)

In [ ]:
IAI.variable_importance(IAI.get_learner(grid_xgb))

### XGBoost regressor

In [ ]:
xgbr = IAI.XGBoostRegressor(random_seed=1, num_estimators=50)
grid_xgbr = IAI.GridSearch(xgbr, max_depth=1:4)

In [ ]:
IAI.fit!(grid_xgbr, X_train_2, y_train_default)

In [ ]:
IAI.score(grid_xgbr,X_test_2, y_test_default,criterion=:mse)

In [ ]:
IAI.variable_importance(IAI.get_learner(grid_xgbr))

## Resample

### XGB regressor

In [ ]:
X_train_resampled = CSV.read("../Data/X_train_resampled.csv", DataFrame)
X_test_resampled = CSV.read("../Data/X_test_resampled.csv", DataFrame)
y_train_resampled = CSV.read("../Data/y_train_resampled.csv", DataFrame)
y_test_resampled = CSV.read("../Data/y_test_resampled.csv", DataFrame)
y_train_resampled = y_train_resampled[!,"TARGET"]
y_test_resampled = y_test_resampled[!,"TARGET"]

In [ ]:
xgbr_resampled = IAI.XGBoostRegressor(random_seed=1, num_estimators=50)
grid_xgbr_resampled = IAI.GridSearch(xgbr_resampled, max_depth=1:4)

In [ ]:
IAI.fit!(grid_xgbr_resampled, X_train_resampled, y_train_resampled)

In [ ]:
IAI.score(grid_xgbr_resampled,X_test_resampled, y_test_resampled,criterion=:mse)

### XGB classifier

In [ ]:
xgb_resampled = IAI.XGBoostClassifier(random_seed=1, num_estimators=50)
grid_xgb_resampled = IAI.GridSearch(xgb_resampled, max_depth=1:4)

In [ ]:
IAI.fit!(grid_xgb_resampled, X_train_resampled, y_train_resampled)

In [ ]:
IAI.score(grid_xgb_resampled,X_test_resampled, y_test_resampled,criterion=:auc)

### ORT

In [ ]:
grid_ort_resampled = IAI.GridSearch(
    IAI.OptimalTreeRegressor(
        random_seed=123,
    ),
    max_depth=1:5,
)

In [ ]:
IAI.fit!(grid_ort_resampled, X_train_resampled, y_train_resampled)

In [ ]:
IAI.score(grid_ort_resampled,X_test_resampled, y_test_resampled,criterion=:mse)

### OCT regression

In [ ]:
grid_oct_r_resampled = IAI.GridSearch(
    IAI.OptimalTreeClassifier(
        random_seed=1,
        minbucket=10,
        refit_learner=IAI.GridSearch(
            IAI.OptimalFeatureSelectionClassifier(),
            sparsity=1:3,
        ),
    ),
    max_depth=1:4,
)

In [ ]:
IAI.fit!(grid_oct_r_resampled, X_train_resampled, y_train_resampled)

In [ ]:
IAI.score(grid_oct_r_resampled,X_test_resampled, y_test_resampled,criterion=:auc)

### OCT

In [ ]:
grid_oct_resampled = IAI.GridSearch(
    IAI.OptimalTreeClassifier(
        random_seed=1),
    max_depth=1:7,
    minbucket=10
)

In [ ]:
IAI.fit!(grid_oct_resampled, X_train_resampled, y_train_resampled)

In [ ]:
IAI.score(grid_oct_resampled,X_test_resampled, y_test_resampled,criterion=:auc)

### ORT RL

In [ ]:
grid_ort_rl_resampled = IAI.GridSearch(
    IAI.OptimalTreeRegressor(
        random_seed=123,
        max_depth=4,
        regression_features=All(),
    ),
    regression_lambda=[0.005, 0.01, 0.05],
)

In [ ]:
IAI.fit!(grid_ort_rl_resampled, X_train_resampled, y_train_resampled)

In [ ]:
IAI.score(grid_ort_rl_resampled,X_test_resampled, y_test_resampled,criterion=:mse)

### Random Forest

In [ ]:
rf_resampled = IAI.RandomForestClassifier(random_seed=1, num_trees = 50)
grid_rf_resampled = IAI.GridSearch(rf_resampled, max_depth=1:4)

In [ ]:
IAI.fit!(grid_rf_resampled, X_train_resampled, y_train_resampled)

In [ ]:
IAI.score(grid_rf_resampled,X_test_resampled, y_test_resampled,criterion=:auc)